In [ ]:
import nibabel as nib
import pydicom
import pyxnat
import re
from pyxnat.core.resources import Scan, CObject, EObject

from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import generate_uid
import tempfile
from datetime import datetime


interface = pyxnat.Interface(
    server='https://mri.lavlab.mcw.edu',
    user='',
    password=''
)

REF_DICOM_SCAN_REGEX = 'ax.*t2.*'
NII_FILE = '/Volumes/Siren/Prostate_data/1101/MRI/Processed/prostate_mask.nii'

# get session for given subject
project_id = 'PROSTATE'
subject_id = '101'
experiment_id = '101_20141111_mr'
subject = interface.select.project(project_id).subject(subject_id)
experiment = subject.experiment(experiment_id)

# get scan to add roi to
scan = None
for scan_id in experiment.scans().get():
    scan = experiment.scan(scan_id)
    assert type(scan) == Scan
    if re.match(REF_DICOM_SCAN_REGEX, scan.attrs.get('type'), re.IGNORECASE):
        break
if scan is None:
    raise Exception(f'No scan found matching regex: {REF_DICOM_SCAN_REGEX}')

dicoms = scan.resource('DICOM')
filename = dicoms.files().get()[0]
dicom_file = dicoms.file(filename)
ref_temp_file = tempfile.NamedTemporaryFile(suffix=".dcm")
ref_dicom = pydicom.read_file(dicom_file.get(ref_temp_file.name))
# Step 1: Read the NIFTI mask
nifti_img = nib.load(NII_FILE)
nifti_data = nifti_img.get_fdata()

# Step 3: Create a new DICOM-SEG object
# print(ref_dicom)
dicomseg = Dataset(ref_dicom)  # Copy the reference DICOM metadata
seg_uid = generate_uid(prefix=None)  # Generate a new UID

# Populate required fields (skipping many details for this example)
dicomseg.SOPClassUID = "1.2.840.10008.5.1.4.1.1.66.4"  # Segmentation Storage
dicomseg.SOPInstanceUID = seg_uid  # Generate a new UID
dicomseg.Modality = "SEG"  # Segmentation
# Step 4: Save as a temporary DICOM file
temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".dcm")
# dicomseg.save_as(temp_file.name)
FileDataset(temp_file.name, dicomseg).save_as(temp_file.name)
dicomseg_file = temp_file.name

# Generate current date and time
current_date = datetime.now().strftime("%Y%m%d")
current_time = datetime.now().strftime("%H%M%S")
# Generate a unique ID and label for the assessor
assessor_id = f"RoiCollection_{current_date}_{current_time}"
assessor_label = f"SEG_{current_date}_{current_time}_S{scan.attrs.get('ID')}"
print(interface.inspect.datatypes('icr:roiCollectionData'))
# Create the assessor
assessor = experiment.assessor(assessor_label)

print(assessor.reference())
assessor.insert(**{ 
    "assessors": 'icr:roiCollectionData', 
    'ID': assessor_id,
    'label': assessor_label,
    'icr:roiCollectionData/collectionType': "SEG",
    'icr:roiCollectionData/subjectID': subject.attrs.get('ID'),
    'icr:roiCollectionData/name': 'Prostate T2 Axial Mask',
    'icr:roiCollectionData/UID': seg_uid,
    
    })

# Upload the DICOMSEG file
resource = assessor.resource('SEG')
resource.insert()
file = resource.file('prostate_mask_T2.dcm')
print(dicomseg_file)
import os
if not os.path.exists(dicomseg_file):
    raise Exception(f"File does not exist: {dicomseg_file}")
file.insert(dicomseg_file, format='DICOM', context="EXTERNAL", description="DICOM Segmentation", labels=['SEG'], tags=[])

